In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *

In [2]:
path='data/lol/'

In [3]:
match = pd.read_csv(path+"matchinfo.csv")
match.head()

,League,Year,Season,Type,blueTeamTag,bResult,rResult,redTeamTag,gamelength,blueTop,...,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp,Address
0,NALCS,2015,Spring,Season,TSM,1,0,C9,40,Dyrus,...,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,LemonNation,Thresh,http://matchhistory.na.leagueoflegends.com/en/...
1,NALCS,2015,Spring,Season,CST,0,1,DIG,38,Cris,...,Irelia,Crumbzz,JarvanIV,Shiphtur,Azir,CoreJJ,Corki,KiWiKiD,Annie,http://matchhistory.na.leagueoflegends.com/en/...
2,NALCS,2015,Spring,Season,WFX,1,0,GV,40,Flaresz,...,Sion,Saintvicious,LeeSin,Keane,Azir,Cop,Corki,BunnyFuFuu,Janna,http://matchhistory.na.leagueoflegends.com/en/...
3,NALCS,2015,Spring,Season,TIP,0,1,TL,41,Rhux,...,Gnar,IWDominate,Nunu,Fenix,Lulu,KEITH,KogMaw,Xpecial,Janna,http://matchhistory.na.leagueoflegends.com/en/...
4,NALCS,2015,Spring,Season,CLG,1,0,T8,35,Benny,...,Sion,Porpoise8,RekSai,Slooshi8,Lulu,Maplestreet8,Corki,Dodo8,Annie,http://matchhistory.na.leagueoflegends.com/en/...


In [4]:
# drop unnessary columns
match.drop(["rResult", "gamelength", "Address"], axis=1, inplace=True)

In [5]:
match.head()

,League,Year,Season,Type,blueTeamTag,bResult,redTeamTag,blueTop,blueTopChamp,blueJungle,...,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp
0,NALCS,2015,Spring,Season,TSM,1,C9,Dyrus,Irelia,Santorin,...,Balls,Gnar,Meteos,Elise,Hai,Fizz,Sneaky,Sivir,LemonNation,Thresh
1,NALCS,2015,Spring,Season,CST,0,DIG,Cris,Gnar,Impaler,...,Gamsu,Irelia,Crumbzz,JarvanIV,Shiphtur,Azir,CoreJJ,Corki,KiWiKiD,Annie
2,NALCS,2015,Spring,Season,WFX,1,GV,Flaresz,Renekton,ShorterACE,...,Hauntzer,Sion,Saintvicious,LeeSin,Keane,Azir,Cop,Corki,BunnyFuFuu,Janna
3,NALCS,2015,Spring,Season,TIP,0,TL,Rhux,Irelia,Rush,...,Quas,Gnar,IWDominate,Nunu,Fenix,Lulu,KEITH,KogMaw,Xpecial,Janna
4,NALCS,2015,Spring,Season,CLG,1,T8,Benny,Gnar,Xmithie,...,CaliTrlolz8,Sion,Porpoise8,RekSai,Slooshi8,Lulu,Maplestreet8,Corki,Dodo8,Annie


In [6]:
print(len(match))

7620


In [7]:
# baseline
print("Blue team winning rate: {:0.2f}%".format(match["bResult"].mean() * 100))

Blue team winning rate: 54.41%


In [8]:
match.columns

Index(['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'bResult',
       'redTeamTag', 'blueTop', 'blueTopChamp', 'blueJungle',
       'blueJungleChamp', 'blueMiddle', 'blueMiddleChamp', 'blueADC',
       'blueADCChamp', 'blueSupport', 'blueSupportChamp', 'redTop',
       'redTopChamp', 'redJungle', 'redJungleChamp', 'redMiddle',
       'redMiddleChamp', 'redADC', 'redADCChamp', 'redSupport',
       'redSupportChamp'],
      dtype='object')

In [9]:
# prepare for champion embeddings
teams = ["blue", "red"]
positions = ["Top", "Jungle", "Middle", "ADC", "Support"]
champ_cols = []

all_champs = set()
for t in teams:
    for p in positions:
        col = t + p + "Champ"
        all_champs |= set(match[col].unique())
        champ_cols.append(col)
all_champs = list(all_champs)
len(all_champs)

140

In [10]:
# prepare for player embeddings
all_players = set()
player_cols = []

for t in teams:
    for p in positions:
        col = t + p
        all_players |= set(match[col].unique())
        player_cols.append(col)
all_players = list(all_players)
len(all_players)

1596

In [11]:
# prepare for team embeddings
all_teams = set()
team_cols = []

for t in teams:
    col = t + "TeamTag"
    all_teams |= set(match[col].unique())
    team_cols.append(col)
all_teams = list(all_teams)
len(all_teams)

247

In [12]:
# map values to unique IDs
other_cols = ["League", "Year", "Season", "Type"]
for col in other_cols:
    unique_vals = match[col].unique()
    val2id = {v: i for i, v in enumerate(unique_vals)}
    match[col] = match[col].map(lambda v: val2id[v])

champ2id = {v: i for i, v in enumerate(all_champs)}
player2id = {v: i for i, v in enumerate(all_players)}
team2id = {v: i for i, v in enumerate(all_teams)}

for champ_col in champ_cols:
    match[champ_col] = match[champ_col].map(lambda v: champ2id[v])

for player_col in player_cols:
    match[player_col] = match[player_col].map(lambda v: player2id[v])

for team_col in team_cols:
    match[team_col] = match[team_col].map(lambda v: team2id[v])

match.head()

,League,Year,Season,Type,blueTeamTag,bResult,redTeamTag,blueTop,blueTopChamp,blueJungle,...,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp
0,0,0,0,0,220,1,51,880,87,128,...,607,134,913,12,51,1,593,13,1209,70
1,0,0,0,0,15,0,53,312,134,1187,...,940,87,1594,25,94,6,164,35,683,78
2,0,0,0,0,172,1,89,919,132,505,...,1046,115,525,94,35,6,405,35,1030,79
3,0,0,0,0,6,0,231,1107,87,1504,...,201,134,1190,122,746,103,768,38,404,79
4,0,0,0,0,106,1,76,797,134,908,...,1224,115,1568,2,1451,103,492,35,1219,78


In [13]:
match.sample(5)

,League,Year,Season,Type,blueTeamTag,bResult,redTeamTag,blueTop,blueTopChamp,blueJungle,...,redTop,redTopChamp,redJungle,redJungleChamp,redMiddle,redMiddleChamp,redADC,redADCChamp,redSupport,redSupportChamp
100,0,0,1,0,220,1,168,880,109,128,...,919,128,1071,53,409,9,1308,97,1476,58
7427,2,1,0,4,24,1,148,816,103,1040,...,81,65,687,2,527,32,1556,86,87,31
4867,5,2,0,1,78,0,133,1212,131,1304,...,32,132,49,112,1573,75,439,0,1528,85
3714,3,1,1,0,20,0,173,779,107,670,...,630,93,582,2,1269,101,417,13,859,31
1407,1,1,0,0,218,0,125,711,28,1327,...,1013,30,1246,37,1519,14,1305,35,1100,123


In [14]:
# prepare features and targets
match = match[['League', 'Year', 'Season', 'Type', 'blueTeamTag', 'redTeamTag',
               'blueTop', 'blueJungle', 'blueMiddle', 'blueADC', 'blueSupport',
               'redTop', 'redJungle', 'redMiddle', 'redADC', 'redSupport',
               'blueTopChamp', 'blueJungleChamp', 'blueMiddleChamp', 'blueADCChamp',
               'blueSupportChamp', 'redTopChamp', 'redJungleChamp', 'redMiddleChamp',
               'redADCChamp', 'redSupportChamp', 'bResult']]

x = match.drop("bResult", axis=1)
y = match['bResult'].astype(np.float32)

val_idxs = get_cv_idxs(len(match))
data = ColumnarModelData.from_data_frame(path, val_idxs, x, y, x.columns.tolist(), 16)

In [15]:
emb_szs = []

for col in other_cols:
    num_unique = len(match[col].unique())
    emb_sz = min(50, num_unique//2)
    emb_szs.append((num_unique, emb_sz))

In [16]:
def get_embedding(no, ne):
    e = nn.Embedding(no, ne)
    e.weight.data.uniform_(-0.05, 0.05)
    return e

class EmbeddingNet(nn.Module):
    def __init__(self, emb_szs, hidden_units, drops):
        super().__init__()
        # embedding layers
        self.embs = nn.ModuleList([get_embedding(no, ne) for no, ne in emb_szs])
        self.champ_embs = get_embedding(len(all_champs), 50)
        self.player_embs = get_embedding(len(all_players), 150)
        self.team_embs = get_embedding(len(all_teams), 50)
        
        # dropout for embedding layers
        self.drop_e = nn.Dropout(0.25)
        
        # neural network
        ne_sum = sum([ne for _, ne in emb_szs])
        hidden_units = [ne_sum + 50*10 + 150*10 + 50*2] + hidden_units
        self.lins = nn.ModuleList([
            nn.Linear(hidden_units[i], hidden_units[i+1]) for i in range(len(hidden_units)-1)
        ])
        
        # dropouts
        self.drops = nn.ModuleList([nn.Dropout(d) for d in drops])
        
        # batch normalization
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(n) for n in hidden_units[1:]
        ])
        
        # output layer
        self.out = nn.Linear(hidden_units[-1], 1)
        
    def forward(self, cats, conts):
        # embedding layers
        x = [e(cats[:, i]) for i, e in enumerate(self.embs)]
        x += [self.team_embs(cats[:, i]) for i in range(4, 6)]
        x += [self.player_embs(cats[:, i]) for i in range(6, 16)]
        x += [self.champ_embs(cats[:, i]) for i in range(16, 26)]

        
        # concatenate embedding layers
        x = torch.cat(x, dim=1)
        x = self.drop_e(x)
        
        # hidden layers
        for l, d, b in zip(self.lins, self.drops, self.bns):
            x = d(b(F.relu(l(x))))
        
        # output layer
        return F.sigmoid(self.out(x))

In [17]:
model = EmbeddingNet(emb_szs, hidden_units=[3000, 1000, 100, 10], drops=[0.5, 0.5, 0.5, 0.5])
# uncomment if train on GPU
# model = model.cuda()

opt = optim.Adam(model.parameters(), 1e-4, weight_decay=1e-5)
fit(model, data, 3, opt, F.binary_cross_entropy)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch      trn_loss   val_loss                               
    0      0.68878    0.688781  
    1      0.658874   0.647778                               
    2      0.612125   0.624086                               



[0.6240857373385291]

In [18]:
def get_accuracy(dl):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for b in dl:
            x, c, y = b[0], b[1], b[2]
            preds = model(x, c)
            preds_labels = to_np(preds >= 0.5).astype(np.float32)
            total += y.size(0)
            correct += (preds_labels == y).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [19]:
train_acc = get_accuracy(data.trn_dl)
val_acc = get_accuracy(data.val_dl)

print("Training set accuracy: {:0.2f}%".format(train_acc))
print("Validation set accuracy: {:0.2f}%".format(val_acc))

Training set accuracy: 73.97%
Validation set accuracy: 66.67%
